In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [2]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.RedNeuronal = nn.Sequential(
            nn.Conv2d(3, 30, (5, 5), stride=2),
            nn.MaxPool2d((1, 1)),
            nn.Conv2d(30, 60, (2, 2), stride=3),
            nn.MaxPool2d((1, 1)),
            nn.ReLU(),
            nn.Conv2d(60, 120, (3, 3), stride=2),
            nn.MaxPool2d((2, 2)),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(9720, 1000),
            nn.Linear(1000, 53))


    def forward(self, x):
        logits = self.RedNeuronal(x)
        return logits
    

    
transform = transforms.Compose([
    transforms.Resize((244, 244)), # Redimensionar a 200x200p
    transforms.ToTensor(), # Convertir imatges a tensors de PyTorch
])

#En este cas les dades venen separades
data_path_train = "/home/itibcn/Desktop/Torch/datasets/cards/train"
data_path_test = "/home/itibcn/Desktop/Torch/datasets/cards/test"

train = datasets.ImageFolder(root=data_path_train, transform=transform)
test = datasets.ImageFolder(root=data_path_test, transform=transform)

dataloader_train = DataLoader(train, batch_size=10, shuffle=True)
dataloader_test = DataLoader(test, batch_size=10, shuffle=True)

output_dim = len(datasets.ImageFolder(root = data_path_test).classes)
bs = 10
epochs = 100

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = NeuralNetwork().to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

print(train.classes)
print(len(train.classes))


def train_loop(dataloader, model, loss_fn, optimizer, bs):
    size = len(dataloader.dataset)
    model.train()

    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)

        #BackPropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * bs + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        test_loss /= num_batches
        correct /= size
        print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


for epoch in range(epochs):
    print("Epoch: " + str(epoch))
    train_loop(dataloader_train, model, loss_fn, optimizer, bs)
    test_loop(dataloader_test, model, loss_fn)

print("Entrenament finalitzat")


['ace of clubs', 'ace of diamonds', 'ace of hearts', 'ace of spades', 'eight of clubs', 'eight of diamonds', 'eight of hearts', 'eight of spades', 'five of clubs', 'five of diamonds', 'five of hearts', 'five of spades', 'four of clubs', 'four of diamonds', 'four of hearts', 'four of spades', 'jack of clubs', 'jack of diamonds', 'jack of hearts', 'jack of spades', 'joker', 'king of clubs', 'king of diamonds', 'king of hearts', 'king of spades', 'nine of clubs', 'nine of diamonds', 'nine of hearts', 'nine of spades', 'queen of clubs', 'queen of diamonds', 'queen of hearts', 'queen of spades', 'seven of clubs', 'seven of diamonds', 'seven of hearts', 'seven of spades', 'six of clubs', 'six of diamonds', 'six of hearts', 'six of spades', 'ten of clubs', 'ten of diamonds', 'ten of hearts', 'ten of spades', 'three of clubs', 'three of diamonds', 'three of hearts', 'three of spades', 'two of clubs', 'two of diamonds', 'two of hearts', 'two of spades']
53
Epoch: 0
loss: 4.003350  [   10/ 7624]

KeyboardInterrupt: 

In [18]:
class RedNeuronal(nn.Module):
    def __init__(self):
        self.Capes = nn.Sequential(
        nn.Conv2d(3, 30, (5,5), stride=2),
        nn.MaxPool2d((2,2)),
        nn.Conv2d(30, 60, (3,3), stride=2),
        nn.MaxPool2d((2, 2)),
        nn.ReLU(),
        nn.Conv2d(60, 120, (2,2), stride=1),
        nn.MaxPool2d((2,2)),
        nn.ReLU(),
        nn.Linear(20000, 100)
        )

    def forward(self, x):
        logits = self.RedNeuronal(x)
        return logits
    
    
    device = ("cuda" if torch.cuda.is_available() else "cpu")    
    data_path_train = "/home/itibcn/Desktop/Torch/datasets/cards/train"
    data_path_test = "/home/itibcn/Desktop/Torch/datasets/cards/test"

    transform = transforms.Compose([
        transforms.Resize((200, 200)),
        transforms.ToTensor()
    ])

    dataset_train = datasets.ImageFolder(data_path_train, transform=True)
    dataset_test = datasets.ImageFolder(data_path_test, transform=True)

    dataloader_train = DataLoader(dataset_train, 10, shuffle=True)
    dataloader_test = DataLoader(dataset_test, 10, shuffle=True)

    model = RedNeuronal().to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0005)
    
    epochs = 10

    def train_loop(dataloader, model, optimizer, loss_fn, bs):
        model.train()
        size = len(dataloader)

        for batch, (X, y) in enumerate(dataloader):
            X.to(device)
            y.to(device)
            pred = model(X)
            loss = loss_fn(pred, y)
            loss.backward()

            optimizer.step()       
            optimizer.zero_grad()

    def test_loop(dataloader, model, loss_fn):
        model.eval()
        test_loss, correct = 0,0
        size = len(dataloader.dataset)
        numbatches = len(dataloader)

        for X,y in dataloader:
            X.to(device)
            y.to(device)
            pred = model(X)
            test_loss = loss_fn(pred, y)
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

        test_loss /= numbatches
        correct /= size
            
             



NameError: name 'RedNeuronal' is not defined